In [2]:
import ee
import geemap
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv(os.path.join(os.path.expanduser("~"),"Desktop","projects","glacier_segmentation","data","glims_data","master_glims_df.csv"))

In [19]:
df.head(20)

,Unnamed: 0,glac_id,src_date,area,db_area,geog_area,geometry,my_area,bboxes
0,137352,G263560E76894S,2001-10-01T00:00:00,165078.00,165079.00,West Antarctica,"POLYGON Z ((-86.622742 -74.908126 0, -86.60214...",165079.189613,"[(-104.88506555000001, -80.40410155000001), (-..."
1,335721,G343222E64409N,1999-01-01T00:00:00,0.00,8091.67,Iceland,"MULTIPOLYGON Z (((-15.37719 64.66761 0, -15.37...",8087.962378,"[(-18.3259125, 63.889517999999995), (-15.06787..."
2,162694,G344790E81287N,2003-07-01T00:00:00,7537.57,7537.75,Randolph Glacier Inventory; Umbrella RC for me...,"POLYGON Z ((-17.271698 81.32674799999999 0, -1...",7537.749681,"[(-19.608540350000002, 80.7810984), (-11.12242..."
3,349467,G298930E65309S,2000-02-21T00:00:00,0.00,7380.90,Various (NSIDC),"POLYGON Z ((-61.70293 -65.536411 0, -61.700284...",7380.898461,"[(-62.721173850000014, -66.2062339), (-59.2828..."
4,325445,G294015E69537S,2002-12-31T00:00:00,7018.37,7018.32,Antarctic Peninsula,"POLYGON Z ((-65.065175 -69.48543600000001 0, -...",7018.287995,"[(-67.27992645, -70.0469613), (-64.29308655000..."
5,223718,G238773E74044S,1956-02-01T00:00:00,6004.85,6004.85,Randolph Glacier Inventory; Umbrella RC for me...,"POLYGON Z ((-123.19689 -73.887737 0, -123.1902...",6004.845548,"[(-123.4321837, -74.4186985), (-118.6099002999..."
6,349618,G329763E69208N,1995-12-16T00:00:00,5792.00,5793.78,East Central Greenland,"POLYGON Z ((-31.144732 69.273087 0, -31.156893...",5793.784620,"[(-31.936098750000003, 68.28904115), (-28.3099..."
7,225046,G260838E72287S,1972-12-04T00:00:00,5260.72,5260.71,Randolph Glacier Inventory; Umbrella RC for me...,"POLYGON Z ((-100.711606 -71.958147 0, -100.711...",5260.708792,"[(-102.75613855, -72.60334189999999), (-95.063..."
8,349641,G329389E69966N,1995-12-16T00:00:00,5168.00,5169.31,East Central Greenland,"POLYGON Z ((-29.967413 70.43454 0, -29.96957 7...",5169.313255,"[(-31.932816350000003, 69.49657295), (-28.9293..."
9,223774,G289107E72447S,1997-09-01T00:00:00,4766.14,4766.14,Randolph Glacier Inventory; Umbrella RC for me...,"POLYGON Z ((-69.107626 -72.144711 0, -69.10735...",4766.142148,"[(-73.39434545000002, -72.7093567), (-68.42528..."


In [39]:
bbeval(df[df.glac_id == "G007026E45991N"].iloc[5,:].bboxes)

[(7.0089054, 45.97916925),
 (7.0466266, 45.97916925),
 (7.0466266, 46.02072175),
 (7.0089054, 46.02072175),
 (7.0089054, 45.97916925)]

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWhx0NyBFL9qRR1n1lUT9OS9owEweHnE-tO9oEcXsnW17VoPRkMOYr4

Successfully saved authorization token.


In [40]:
bbox = [(7.0089054, 45.97916925),
 (7.0466266, 45.97916925),
 (7.0466266, 46.02072175),
 (7.0089054, 46.02072175),
 (7.0089054, 45.97916925)]

begDate = poly_date-delta
endDate = poly_date+delta

bbox = one_glacier.bboxes
bbox = eval(list(bbox)[0])
region = ee.Geometry.Polygon(list(bbox))

colL8 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA')
colL8 = col.filterDate(begDate,endDate)
colL8 = col.filterBounds(region)

withCloudiness = colL8.map(algorithm=cloudscore)
colSelectedL8 = withCloudiness.filter(ee.Filter.lt('cloud', cloud_tol))
colSelectedL8 = colSelectedL8.select(['B1','B2','B3','B4','B5','B7'])
colListL8 = colSelectedL8.toList(colL8.size())
colSizeL8 = colSelectedL8.size().getInfo()

print("Number of Images: ",colSizeL8)

out_dir = os.path.join(os.path.expanduser('~'),"Desktop","projects","glacier_segmentation")
filename = "landsat.tif" 
image = ee.Image(colListL8.get(0))
filename = "my_ee_raster"
DEM = ee.Image("NASA/NASADEM_HGT/001")
DEM = DEM.clip(region)
DEM = DEM.select(['elevation'])

NameError: name 'glims_id' is not defined